### (0) IMPORT

In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#BORROWING KAISHENGS CLIENT SECRET AND ID THANKS KS BB PLS DONT DISABLE YA
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

### (1) Playlist extractor - Make sure you do not already have a file called "User_Playlist.csv", else it will be replaced.

In [ ]:
import time

# def extract_playlistID(playlists,find_playlist):
#     while True:
#         for playlist in playlists['items']:
#         ##Loop and return playlistID if found
#             if playlist["name"] == find_playlist:
#                 return playlist['id']
#         ##Show next 50 songs
#         if playlists['next']:
#             playlists = sp.next(playlists)
#         else:
#             return None
        
def extract_trackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_nexttrackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_songInfo(song):
    info = sp.track(song)
    ##Define features u wanna extract here
    id = song
    songName = info["name"]
    albumName = info["album"]["name"]
    albumID = info["album"]["id"]
    artists = info["artists"][0]["name"]
    for artist in info["artists"]:
        artist_genre = sp.artist(artist["id"])
        genre = artist_genre["genres"]
    explicit = info["explicit"]
    releaseDate = info["album"]["release_date"]
    return [id,songName,albumName,albumID,artists,genre,explicit,releaseDate]

def extract_features(song):
    info = sp.audio_features(song)[0]
    ##Define features u wanna extract here
    danceability = info["danceability"]
    energy = info["energy"]
    key = info["key"]
    loudness = info["loudness"]
    mode = info["mode"]
    speechiness = info["speechiness"]
    acousticness = info["acousticness"]
    instrumental = info["instrumentalness"]
    liveness = info["liveness"]
    valence = info["valence"]
    tempo = info["tempo"]
    duration_ms = info["duration_ms"]
    time_signature = info["time_signature"]
    return [danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,
            valence,tempo,duration_ms,time_signature]

#Auth Stuff
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

##INDICATE THE PLAYLIST U WANNA ADD FROM THE USER (IN THIS CASE SPOTIFY IS THE USER AND TOP 50 IS THE PLAYLIST)
# user = "spotify"
# playlists = sp.user_playlists(user, limit=50)

## Hide this if you are inserting your own playlist ID
# for find_playlist in playlists_name:
#     found_ID = extract_playlistID(playlists, find_playlist)
#     if found_ID != None:
#         playlists_ID.append(found_ID)
    
playlists_name = []
playlists_ID = []

## Ask to insert playlist ID
while True:
    url = input("Insert Spotify Playlist ID (Else type 'done'): ")
    if url == "done":
        break
    else:
        playlists_ID.append(url)
    
# Get Playlist Name
print("Obtaining the Following Playlists: ")
for i in playlists_ID:
    playlist_info = sp.playlist(i)
    name = playlist_info["name"]
    print(name)
    playlists_name.append(name)

##Get Info in Playlist with PlaylistID 
tracksID = {}
for i in range(len(playlists_ID)):
    track_list = []
    playlist_tracks = sp.playlist(playlists_ID[i])
    playlist_tracks = playlist_tracks["tracks"]
    track_list = extract_trackID(playlist_tracks)
    for page in range(0,2):
        if playlist_tracks['next']:
            print("More than 100 Songs Detected. WAITT")
            time.sleep(30)
            print("Looking at next 100 songs")
            playlist_tracks = sp.next(playlist_tracks)
            track_list += extract_nexttrackID(playlist_tracks)
    tracksID[playlists_name[i]] = track_list


##Get Other Track Details
results_tocsv = []
results_tocsv.append(["id","songName","albumName","albumID","artist","genre","explicit","releaseDate",
                     "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                    "valence","tempo","duration_ms","time_signature"])
for i in playlists_name:  
    count = 0
    for song in tracksID[i]:
        time.sleep(1)
        song_info = extract_songInfo(song)
        ##Get Audio Features
        song_features = extract_features(song)
        song_info.extend(song_features)
        results_tocsv.append(song_info)
        count += 1
        
        if count == 100:
            print("Trying not to let spotify kill me. 30s wait :D")
            time.sleep(30)
            count = 0

##Use pandas to convert to dataframe
df = pd.DataFrame(results_tocsv[1:], columns = results_tocsv[0])

##Export to CSV
file_name = "User_Playlist.csv"
df.to_csv(file_name, encoding='utf_8_sig')
print("20s Cooldown...")
time.sleep(20)

### (2) FUNCTIONSSS

In [12]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF, feature_weights):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * feature_weights['key']
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * feature_weights['mode']

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

    # Apply weight to each float column
    for col in floats_scaled.columns:
        floats_scaled[col] *= feature_weights.get(col, 1.0)

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final


feature_weight = {
    "danceability": 0.6,
    "energy": 0.7,
    "key": 0.1,
    "loudness": 0.8,
    "mode": 0.1,
    "speechiness": 0.5,
    "acousticness": 0.3,
    "instrumentalness": 0.5,
    "liveness": 0.3,
    "valence": 0.7,
    "tempo": 0.6,
    "type": 0.1
}

# feature_weight = {
#     "danceability": 1,
#     "energy": 1,
#     "key": 1,
#     "loudness": 1,
#     "mode": 1,
#     "speechiness": 1,
#     "acousticness": 1,
#     "instrumentalness": 1,
#     "liveness": 1,
#     "valence": 1,
#     "tempo": 1,
#     "type": 1
# }

def generate_rec(databaseDF, database_vector, user_vector, genre):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)

    #Drop rows with different genre from top 3 genres
    databaseDF = databaseDF[(databaseDF["genre"] == genre)]

    rec_top5 = databaseDF.sort_values('sim',ascending = False).head()
    return rec_top5

    #KEEP FOR FUTURE USE IF NEEDED
    # final_reco = []
    
    # # # Drop rows with different genre from top 3 genres
    # # rec_top5 = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1]) | (databaseDF["genre"] == genre_top3[2])]

    # #Sort and recommend top 5 with same genres
    # pre_rec_ordered = databaseDF.sort_values('sim',ascending = False).head(20)

    # #Find genre for each db song in order, and append first 5 songs once genre match is found
    # for i in range(20):
    #     track_data = sp.track(pre_rec_ordered["id"].iloc[i])

    #     artist_ids = []
    #     for artist in track_data["artists"]:
    #         artist_ids.append(artist["id"])

    #         artists_data = sp.artists(artist_ids)

    #     genres = []
    #     for artist in artists_data["artists"]:
    #         ls_art_genre = artist["genres"]

    #         for j in ls_art_genre: #['funk']
    #             genres.append(j)

    #     genres = set(genres) # removes duplicates

    #     #UNCOMMENT LINE BELOW to show top songs even if none of them share same genre (WHEN THERE ARE NO SONGS RECOMMENDED)
    #     # print([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres]) 

    #     for g in genres:
    #         if g in genre_top3:
    #             final_reco.append([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres])
                
    # return final_reco
    



### (3) NORMALISING Genre Names
If METHOD1 does not work for particular song, do METHOD2 so on and so forth <br><br>
METHOD1 - Find User Songs in Dataset and Copy Over <br>
METHOD2 - Webscrap from get_genre.com which genre name is generic and similar to dataset<br>
METHOD3 - SPOTIFYAPI


<hr/><br>
METHOD2 - Webscraping Function: REQUIRES SELENIUM AND BS4 LIBRARY. FOR SELENIUM NEEDS TO INSTALL CHROMEDRIVERS TOO from below: <br>


[CHROMEDRIVERS](https://googlechromelabs.github.io/chrome-for-testing) 

In [ ]:
# # !pip install bs4
# !pip install selenium==3.141.0 


In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

# chromedriver = '/Users/ercong/Documents/yippee/chromedriver'

def scrap_genre(artistName):

    ##URL of website to scrap
    url = f"https://www.getgenre.com/artist/{artistName}"
    ##CHANGE PATH TO PATH OF DOWNLOADED CHROMEDRIVER
    # service = Service(executable_path=chromedriver)
    driver = webdriver.Chrome(executable_path='/Users/kaisheng/Documents/chromedriver')

    # WAIT FOR BUTTON TAG TO LOAD BEFORE CONTINUING
    driver.implicitly_wait(30)
    driver.get(url)

    classtext = "MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium css-x3ahaf"
    button = driver.find_element_by_xpath(f"//button[contains(@class, '{classtext}')]")
    time.sleep(2)
    # Parse the HTML content of the button with BS4
    html = button.get_attribute("outerHTML")
    driver.close()
    driver.quit()

    #Use BS4 to read HTML segement and extract Genre Text
    soup = BeautifulSoup(html, "html.parser")
    try:
        # Find html button tag with class to obtained genre text
        genre = soup.find('button', class_=classtext).get_text()
        return genre.lower()
    except:
        pass


In [7]:
##Import User Playlist
userDF = pd.read_csv("User_Playlist.csv")
userDF = userDF.drop(columns=["Unnamed: 0"])
##Import Dataset
df = pd.read_csv("spotify_data_cleaned.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna()

#Extract all genres from dataset
datasetGenres_list = []
for i in df["genre"]:
    if i not in datasetGenres_list:
        datasetGenres_list.append(i)


song_genres_list = []
##Find Playlist Songs in Database and replace genre

for index, row in userDF.iterrows():
    id = row["id"]
    artistName = row["artist"]

    ##List to include all genres from one song
    one_song_genre = []

    ##variable to check if song in database
    dataset_genre = df[df["id"] == id]

    if not dataset_genre.empty:
        ##METHOD1 - Copying Dataset genre to User Dataset
        one_song_genre = dataset_genre["genre"].to_list()
    else:
        ##METHOD2 - Web Scraping
        genre = scrap_genre(artistName)
        if genre != "":
            one_song_genre.append(genre)

    ##CHECKS IF METHOD 1/2 managed to obtain genre data
    song_genres_list.append(one_song_genre)

userDF["genre"] = [genre[0] for genre in song_genres_list]
userDF



,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,24yrJJkmVUMkJHvCaR85BN,Snooze,Snooze,0L632gxhMbscW5mY9ewbOq,SZA,contemporary r&b,True,2023-08-25,0.571,0.5520,...,-7.231,1,0.1300,0.1400,0.000000,0.1090,0.3930,143.024,201800,4
1,0otRX6Z89qKkHkQ9OqJpKt,Car's Outside,YOU,0cm9LOQUBK3JUgyoRj238z,James Arthur,pop,False,2019-10-18,0.338,0.5620,...,-7.181,1,0.0290,0.0390,0.000000,0.1070,0.2590,149.907,248373,4
2,6ErCfbbI9BG4bK2djjKUvP,i wish you cheated,i wish you cheated,1tf89mHJkN2HVQ4xmdUUqy,Alexander Stewart,pop,False,2023-08-09,0.348,0.3810,...,-7.688,1,0.0552,0.2040,0.000000,0.1330,0.4940,84.896,190352,3
3,3vZk7OAUjMtVDNC852aNqi,Oceans & Engines,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,NIKI,contemporary r&b,False,2022-07-08,0.395,0.3090,...,-8.919,1,0.0330,0.6370,0.000022,0.0931,0.0673,142.012,336001,3
4,1kuGVB7EU95pJObxwvfwKS,vampire,GUTS,1xJHno7SmdVtZAtXbdbDZp,Olivia Rodrigo,pop,True,2023-09-08,0.511,0.5320,...,-5.745,1,0.0578,0.1770,0.000000,0.2910,0.3500,138.005,219724,4
5,6wf7Yu7cxBSPrRlWeSeK0Q,What Was I Made For? [From The Motion Picture ...,What Was I Made For? [From The Motion Picture ...,3AafSrFIbJPH6BJHiJm1Cd,Billie Eilish,pop,False,2023-07-13,0.444,0.0911,...,-17.665,1,0.0307,0.9590,0.000001,0.0980,0.1420,78.403,222370,4
6,3GYlZ7tbxLOxe6ewMNVTkw,TV,Guitar Songs,1YPWxMpQEC8kcOuefgXbhj,Billie Eilish,pop,False,2022-07-21,0.412,0.2360,...,-15.159,0,0.0379,0.8300,0.006530,0.1470,0.1190,140.214,281381,4
7,7hDVYcQq6MxkdJGweuCtl9,ocean eyes,dont smile at me,7fRrTyKvE4Skh93v97gtcU,Billie Eilish,pop,False,2017-12-22,0.511,0.3630,...,-7.650,0,0.0410,0.8160,0.031700,0.0840,0.1690,144.892,200379,4
8,4VuS959DSpr82t3qBqCrWG,Supermarket Flowers,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,Ed Sheeran,pop,False,2017-03-03,0.589,0.2420,...,-10.517,1,0.0442,0.9140,0.000000,0.0887,0.2570,89.749,221107,4
9,1ZMiCix7XSAbfAJlEZWMCp,Falling,Fine Line,7xV2TzoaVc0ycW7fwBwAml,Harry Styles,pop,False,2019-12-13,0.567,0.2670,...,-6.502,1,0.0299,0.8390,0.000001,0.0890,0.0592,110.011,240133,4


### Clustering Gang Gang

In [8]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming userDF is an existing dataframe
df = userDF.copy()

# features for clustering
X = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumental', 'liveness', 'valence', 'tempo']]


#choose how many clusters u want
number_clusters = 3


# Perform KMeans clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0)
df['cluster'] = kmeans.fit_predict(X)

df


# clusters = {}
# topgenre_clusters = []
# allgenres = []

# # Output each cluster to a separate dataframe
# for cluster_num in range(number_clusters):
#     cluster_df = df[df['cluster'] == cluster_num].copy()  # Make a copy to avoid modifying the original dataframe
#     clusters[cluster_num] = cluster_df

#     common_genre = cluster_df['genre'].mode()[0]  # Find the most common genre in the cluster
    
#     #newlist for top genres in each cluster
#     topgenre_clusters.append(common_genre)
#     allgenres = df['genre'].unique().tolist()


/Users/kaisheng/miniconda3/envs/Spotify/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature,cluster
0,24yrJJkmVUMkJHvCaR85BN,Snooze,Snooze,0L632gxhMbscW5mY9ewbOq,SZA,contemporary r&b,True,2023-08-25,0.571,0.5520,...,1,0.1300,0.1400,0.000000,0.1090,0.3930,143.024,201800,4,0
1,0otRX6Z89qKkHkQ9OqJpKt,Car's Outside,YOU,0cm9LOQUBK3JUgyoRj238z,James Arthur,pop,False,2019-10-18,0.338,0.5620,...,1,0.0290,0.0390,0.000000,0.1070,0.2590,149.907,248373,4,0
2,6ErCfbbI9BG4bK2djjKUvP,i wish you cheated,i wish you cheated,1tf89mHJkN2HVQ4xmdUUqy,Alexander Stewart,pop,False,2023-08-09,0.348,0.3810,...,1,0.0552,0.2040,0.000000,0.1330,0.4940,84.896,190352,3,1
3,3vZk7OAUjMtVDNC852aNqi,Oceans & Engines,Oceans & Engines,2va673nk2JXgCxJeTiZdM9,NIKI,contemporary r&b,False,2022-07-08,0.395,0.3090,...,1,0.0330,0.6370,0.000022,0.0931,0.0673,142.012,336001,3,0
4,1kuGVB7EU95pJObxwvfwKS,vampire,GUTS,1xJHno7SmdVtZAtXbdbDZp,Olivia Rodrigo,pop,True,2023-09-08,0.511,0.5320,...,1,0.0578,0.1770,0.000000,0.2910,0.3500,138.005,219724,4,0
5,6wf7Yu7cxBSPrRlWeSeK0Q,What Was I Made For? [From The Motion Picture ...,What Was I Made For? [From The Motion Picture ...,3AafSrFIbJPH6BJHiJm1Cd,Billie Eilish,pop,False,2023-07-13,0.444,0.0911,...,1,0.0307,0.9590,0.000001,0.0980,0.1420,78.403,222370,4,1
6,3GYlZ7tbxLOxe6ewMNVTkw,TV,Guitar Songs,1YPWxMpQEC8kcOuefgXbhj,Billie Eilish,pop,False,2022-07-21,0.412,0.2360,...,0,0.0379,0.8300,0.006530,0.1470,0.1190,140.214,281381,4,0
7,7hDVYcQq6MxkdJGweuCtl9,ocean eyes,dont smile at me,7fRrTyKvE4Skh93v97gtcU,Billie Eilish,pop,False,2017-12-22,0.511,0.3630,...,0,0.0410,0.8160,0.031700,0.0840,0.1690,144.892,200379,4,0
8,4VuS959DSpr82t3qBqCrWG,Supermarket Flowers,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,Ed Sheeran,pop,False,2017-03-03,0.589,0.2420,...,1,0.0442,0.9140,0.000000,0.0887,0.2570,89.749,221107,4,1
9,1ZMiCix7XSAbfAJlEZWMCp,Falling,Fine Line,7xV2TzoaVc0ycW7fwBwAml,Harry Styles,pop,False,2019-12-13,0.567,0.2670,...,1,0.0299,0.8390,0.000001,0.0890,0.0592,110.011,240133,4,2


In [9]:
clusters = {}
topgenre_clusters = []
allgenres = []

# Output each cluster to a separate dataframe
for cluster_num in range(number_clusters):
    cluster_df = df[df['cluster'] == cluster_num].copy()  # Make a copy to avoid modifying the original dataframe
    clusters[cluster_num] = cluster_df

    common_genre = cluster_df['genre'].mode()[0]  # Find the most common genre in the cluster
    
    #newlist for top genres in each cluster
    topgenre_clusters.append(common_genre)
    allgenres = df['genre'].unique().tolist()
topgenre_clusters

['pop', 'pop', 'pop']

### (5) Extract Both User and Dataset Csv

In [10]:
#Extract Database CSV
databaseDF = pd.read_csv("spotify_data_cleaned.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]

#Create new feature/column of artist_songName
databaseDF["artist_songName"] = databaseDF["artist"] + "_" + databaseDF["songName"]

##Change all values to lowercase
databaseDF["artist_songName"] = databaseDF["artist_songName"].str.lower()

#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)
databaseDF

,id,artist,songName,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,artist_songName
0,53QF56cjZA9RTuuMZDrSA6,Jason Mraz,I Won't Give Up,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.69400,0.000000,0.1150,0.139,133.406,jason mraz_i won't give up
1,1s8tP3jP4GZcyHDsjvw218,Jason Mraz,93 Million Miles,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.47700,0.000014,0.0974,0.515,140.182,jason mraz_93 million miles
2,7BRCa8MPiyuvr2VU3O9W0F,Joshua Hyslop,Do Not Let Me Go,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.33800,0.000050,0.0895,0.145,139.832,joshua hyslop_do not let me go
3,63wsZUhUZLlh1OsyrZq7sz,Boyce Avenue,Fast Car,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.80700,0.000000,0.0797,0.508,204.961,boyce avenue_fast car
4,6nXIYClvJAfi6ujLiKqEq8,Andrew Belle,Sky's Still Blue,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.07260,0.019300,0.1100,0.217,171.864,andrew belle_sky's still blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027751,1tpOChGX8v34yHVukGk8DG,Akkarai Subhalakshmi,Sravanam - Madhyamavathi - Adi (Live),indian,0.419,0.748,2,-10.657,1,0.1940,0.67400,0.000022,0.0800,0.827,171.229,akkarai subhalakshmi_sravanam - madhyamavathi ...
1027752,2NU6Ne7jNlunEkmat52m8W,Fossils,Guru,indian,0.576,0.787,2,-6.503,1,0.0459,0.00313,0.000621,0.3780,0.593,112.010,fossils_guru
1027753,4w11WNonK6ZRGyWVWLWM8n,Lalgudi Vijayalakshmi,Nidhichala,indian,0.534,0.314,4,-13.593,1,0.0533,0.95100,0.762000,0.2890,0.490,138.522,lalgudi vijayalakshmi_nidhichala
1027754,1ypYdGD0dz8bRLUF9kdG1L,Lalgudi Vijayalakshmi,Govardhana,indian,0.494,0.319,4,-14.487,0,0.0536,0.95800,0.784000,0.1010,0.398,150.335,lalgudi vijayalakshmi_govardhana


ITERATE THROUGH ALL THE CLUSTERS

In [14]:

result = pd.DataFrame()  # Initialize finalrec as an empty DataFrame


for i in range(len(topgenre_clusters)):
    clusternumber = i
    genre = topgenre_clusters[i]
    #pullout existing user playlist clusters
    playlistDF = clusters[clusternumber]

    #Selects columns that we want
    playlistDF = playlistDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
            "valence","tempo",]]

    #Create new feature/column of artist_songName to remove duplicate later on
    playlistDF["artist_songName"] = playlistDF["artist"] + "_" + playlistDF["songName"]
    ##Change all values to lowercase
    playlistDF["artist_songName"] = playlistDF["artist_songName"].str.lower()

    #Check for duplicates in user Playlist
    playlistDF = playlistDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)






    # Get the column names of each DataFrame
    playlist_columns = set(playlistDF.columns)
    database_columns = set(databaseDF.columns)

    # Find columns in playlistDF that are not in databaseDF
    columns_in_playlist_not_in_database = playlist_columns - database_columns

    # Find columns in databaseDF that are not in playlistDF
    columns_in_database_not_in_playlist = database_columns - playlist_columns




    #Merge user + dataset dataframe to normalise 
    #Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively

    #Group the dataframe as we gonna split it again alter
    databaseDF["type"] = "Dataset"
    playlistDF["type"] = "User"


    #Merge the 2 datasets together
    combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)

    #Check for duplicates between user and Database
    combinedDF = combinedDF.drop_duplicates(subset=["artist_songName"], keep="last",ignore_index= True)

    ##Update databaseDF with removed songs from user Playlist
    databaseDF = combinedDF[(combinedDF["type"]== "Dataset")]

    ##Normalise and get Vectors for Dataset + User
    normalised_vector = get_features_database(combinedDF, feature_weight)

    ##Seperate User from databaseDF 
    database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
    user_vector = normalised_vector[normalised_vector["type"] == "User"]

    #Drop "type" column
    database_vector = database_vector.drop(columns="type")
    user_vector = user_vector.drop(columns="type")
    databaseDF = databaseDF.drop(columns=["type"])







    ##Single Vector Creation
    final_user_vector_list = []
    for i in user_vector.columns:
        final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

    #Putting into a vector dataframe
    final_user_vector = pd.DataFrame(columns=user_vector.columns,)
    final_user_vector.loc[0] = final_user_vector_list

    final_user_vector



    def generate_rec(databaseDF, database_vector, user_vector, genre, finalrec):
        # Cosine Similarity
        databaseDF["sim"] = cosine_similarity(database_vector, user_vector)

        # Drop rows with different genre from top 3 genres
        databaseDF = databaseDF[databaseDF["genre"].astype(str) == str(genre)]

        rec_top5 = databaseDF.sort_values('sim', ascending=False).head()
        num_rows = rec_top5.shape[0]

        rec_top5['cluster'] = clusternumber  # Assuming clusternumber is defined elsewhere
        df_concatenated = pd.concat([finalrec, rec_top5], ignore_index=True)
        return df_concatenated  # Return the concatenated DataFrame





    genre = topgenre_clusters[clusternumber]
    final = generate_rec(databaseDF, database_vector, final_user_vector, genre, result) 
    result = final
    

result


/var/folders/gw/kfvy784x0szfx_w98v92059w0000gp/T/ipykernel_9172/1645196379.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlistDF["artist_songName"] = playlistDF["artist"] + "_" + playlistDF["songName"]
/var/folders/gw/kfvy784x0szfx_w98v92059w0000gp/T/ipykernel_9172/1645196379.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlistDF["artist_songName"] = playlistDF["artist_songName"].str.lower()
/var/folders/gw/kfvy784x0szfx_w98v92059w0000gp/T/ipykernel_9172/1645196379.py:44: SettingWithCopyW

,id,artist,songName,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,artist_songName,sim,cluster
0,5cjBiZxqUDm5wtc3NeePEl,Jay Chou,紅塵客棧,pop,0.432,0.468,0,-8.602,1,0.0259,0.387,0.000148,0.1850,0.274,180.016,jay chou_紅塵客棧,0.992518,0
1,3adnLFXKO5rC1lhUNSeg3N,Dido,White Flag,pop,0.509,0.538,5,-6.835,1,0.0379,0.306,0.000005,0.0910,0.284,170.111,dido_white flag,0.991042,0
2,3w1P8nwsVi9vxrGvxYLzAC,LÉON,Hope Is A Heartache,pop,0.468,0.495,5,-6.634,1,0.0822,0.429,0.000000,0.3050,0.235,171.400,léon_hope is a heartache,0.990961,0
3,6rCyXpDwlMf1GH2qlpjJWv,Camila,Abrázame,pop,0.458,0.472,6,-7.183,1,0.0478,0.417,0.000000,0.1040,0.322,150.936,camila_abrázame,0.990928,0
4,5bZeugrr7oIATt1CLJuhT4,James TW,Incredible,pop,0.556,0.589,0,-6.209,1,0.0431,0.306,0.000000,0.1470,0.264,180.942,james tw_incredible,0.990781,0
5,6WoQzK5Exbfcmow1AoNRkl,JJ Lin,可惜沒如果,pop,0.486,0.424,0,-5.755,1,0.0262,0.617,0.000000,0.1160,0.286,79.989,jj lin_可惜沒如果,0.993700,1
6,2VxQqF52M7IPkiLd4XmTNi,Kali Uchis,Como Te Quiero Yo,pop,0.501,0.427,0,-8.715,1,0.0312,0.675,0.000004,0.2970,0.320,88.872,kali uchis_como te quiero yo,0.992285,1
7,7uQRJoERlXf7ENerIQ12NS,Lamp,6 Goushitsu,pop,0.513,0.432,0,-9.985,1,0.0265,0.781,0.000012,0.1270,0.333,93.000,lamp_6 goushitsu,0.992120,1
8,4YNrBVo8fv6qiTNwHLVDai,Summer Walker,Potential,pop,0.530,0.370,1,-7.416,1,0.0410,0.572,0.000000,0.0765,0.291,72.732,summer walker_potential,0.991746,1
9,0Om9WAB5RS09L80DyOfTNa,Taylor Swift,Tim McGraw,pop,0.580,0.491,0,-6.462,1,0.0251,0.575,0.000000,0.1210,0.425,76.009,taylor swift_tim mcgraw,0.991333,1
